# Imports

In [432]:
import xml
import xml.etree.ElementTree as ET

import string
import unidecode
import re

import spacy
from spacy.lang.nl.stop_words import STOP_WORDS

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [405]:
nlp = spacy.load('nl_core_news_lg')
stops_words = STOP_WORDS

## Exploring Data

In [406]:
year = ['1986', '1994', '1998']

#### Extracting All Themes

In [466]:
# taxonomy_xml = ET.parse(f'data/Political_election_manifestos/taxonomy.{year[1]}.xml')
taxonomy_xml = xml.dom.minidom.parse(f'data/Political_election_manifestos/taxonomy.{year[1]}.xml')

Simple theme taxonomy exrtaction method: take all themes and ignore the relationship between themes.

In [470]:
# SIMPLE THEME COLLECTION
taxonomy = set()
for theme in taxonomy_xml.getElementsByTagName('theme'):
    taxonomy.add(theme.getAttribute('id'))

More complex list that captures the relation between themes (not complete yet).

In [ ]:
taxonomy = {}
for theme in taxonomy_xml.getroot():
    tax = {}
    # print(theme)
    for sub in theme:
        for subsub in sub:
            if sub.tag == 'related_themes':
                print('RELATED', subsub.get('id'))
            else:
                print('REFERRES', subsub.text)
            # print(subsub.text)

#### Extracting Text Data

In [409]:
document = xml.dom.minidom.parse(f'data/Political_election_manifestos/VP_{year[0]}.party-topicnr-content.xml')

In [410]:
def apply_text_preprocessing(text: str):
    # Strip + Lower case
    text = text.strip().lower()
    # Remove digits
    text = unidecode.unidecode(text)
    # Remove Punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #  Remove stop words and Lemmatisation
    text = ' '.join([token.lemma_ for token in nlp(text) if token.is_stop == False and token.text != ' '])

    return text

In [411]:
chapters = {}
for chapter in document.firstChild.getElementsByTagName('chapter'):
    party = chapter.getAttribute('party')
    # print(chapter.getAttribute('party'))
    par = {}
    for paragraph in chapter.getElementsByTagName('p'):
        id = paragraph.getAttribute('id')
        # print(paragraph.getAttribute('id'))
        
        paragraph_value = paragraph.childNodes[2].nodeValue
        # paragraph_value = apply_text_preprocessing(paragraph_value)
        # print(paragraph_value)

        paragraph_themes = [theme.getAttribute('id') for theme in paragraph.getElementsByTagName('theme')]
        # print(paragraph_themes)

        par[id] = {'p': paragraph_value, 'themes': paragraph_themes}
    chapters[party] = par

In [412]:
X, y = [], []
for chapter in chapters.keys():
    for text in chapters[chapter].keys():
        # if chapters[chapter][text]['p'] != '':
        X.append(chapters[chapter][text]['p']) 
        y.append(chapters[chapter][text]['themes'])

In [413]:
print(X[0])


 Samen werken voor morgen Het CDA heeft zich in de voorbije regeerperiode metterdaad ingezet om de klemgereden samenleving in het goede spoor te brengen . Dat was heel wat moeilijker dan in ons land werd voorzien . Daar kwamen (en komen) ook soms diep-ingrijpende maatregelen aan te pas. Maar het is mede aan dat gevoerde beleid te danken dat het deze vier jaar lukte om nieuw uitzicht te scheppen: de stijging van de werkloosheid kwam tot staan, de overheidsuitgaven worden beter beheerst. Bemoedigend is vooral dat in de maatschappij de bereidheid toenam om eigen verantwoordelijkheid te dragen. Wij staan voor dat beleid. Daarmee werd de basis gelegd voor een gemeenschappelijke krachtsinspanning van overheid en samenleving. Zo'n basis is nodig willen we de problemen-van-morgen de baas worden. Vol overtuiging roept dit verkiezingsprogram dan ook op te blijven ijveren voor een maatschappij waarin we met elkaar, vÃ³Ã³r elkaar opkomen. Een maatschappij die, overeenkomstig de door het CDA voor 

In [414]:
print(len(X))
print(len(y))

797
797


In [415]:
print(type(X))
print(type(y))

<class 'list'>
<class 'list'>


### TF_IDF

In [416]:
# play around with n_gram and max_features
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2', max_features = 10000)

X_tfidf = vectorizer.fit_transform(X)

### Train-Test Split

In [417]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.33, random_state=42)

In [418]:
print(type(X_train))
print(type(y_train))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'list'>


# Models